# Python Learning Weekend Sprint

In [1]:
print('test')

test


---

# For loops

In [2]:
arr = [1,2,"test", 3.5]

for item in arr:
    print (item)

1
2
test
3.5


# Unpacking in for loop

In [5]:
arr = [1,2,"test", 3.5] #Can't be unpacked

for idx,item in arr:  #----> idx Not allowed
    print (idx ,":",item)

TypeError: cannot unpack non-iterable int object

# To make it unpackable or iteratable

In [6]:
arr = [(1,2),("test", 3.5)]

for idx,item in arr:
    print (idx, ":", item)

1 : 2
test : 3.5


# Then how to get index in array/ list elements?

In [9]:
arr = [1,2,"test", 3.5] #Can't be unpacked, hence lets enumurate

for idx,item in enumerate(arr,1):  #----> Now allowed
    print (idx ,":",item)

1 : 1
2 : 2
3 : test
4 : 3.5


In [18]:

list  = [1,'two',3,'four',5,'six']

# print(list.pop(0))
list.pop(0)
list.append(7)
list.insert(6,8)
list

['two', 3, 'four', 5, 'six', 7, 8]